## Benchmark

In [1]:
import mikan
import SimpleITK as sitk
import time
from medpy.metric import dc, hd, hd95, assd

## Load Data

In [2]:
gt = sitk.ReadImage(rf"..\data\patients_26_ground_truth.nii.gz", sitk.sitkUInt8)
pred = sitk.ReadImage(rf"..\data\patients_26_segmentation.nii.gz", sitk.sitkUInt8)

gt_arr = sitk.GetArrayFromImage(gt)
pred_arr = sitk.GetArrayFromImage(pred)

### DSC

In [3]:
# mikan: DSC
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
dsc = evaluator.labels([1,2,3,4,5]).metrics("dsc")
mikan_costs = time.time() - t

# medpy: DSC
t = time.time()
for i in (1,2,3,4,5):
    dsc = dc(pred_arr == i, gt_arr == i)
medpy_costs = time.time() - t

print(f"DSC: {medpy_costs / mikan_costs :.2f}x faster")


DSC: 7.62x faster


### HD

In [4]:
# mikan: HD
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
hausdorff_distance = evaluator.labels([1,2,3,4,5]).metrics("HD")
mikan_costs = time.time() - t
print(f"Mikan has calculated Hausdorff distance and cost {mikan_costs:.2f} s.")
print(f"Let's waiting for medpy, be patient for a while...")

# medpy: HD
t = time.time()
for i in (1,2,3,4,5):
    hausdorff_distance = hd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
medpy_costs = time.time() - t

print(f"HD: {medpy_costs / mikan_costs :.2f}x faster")

Mikan has calculated Hausdorff distance and cost 11.18 s.
Let's waiting for medpy, be patient for a while...
HD: 52.38x faster


### HD/HD95/ASSD

In [5]:
# mikan: Distances
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
mikan_distances = evaluator.labels([1,2,3,4,5]).metrics(["hd", "hd95", "assd"])
mikan_costs = time.time() - t
print(mikan_distances)
print(f"Mikan has calculated distance and cost {mikan_costs:.2f} s.")
print(f"Let's waiting for medpy, be patient for a while...")

# medpy: Distances
t = time.time()
medpy_results = {}
for i in (1,2,3,4,5):
    hd_ = hd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    hd95_ = hd95(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    assd_ = assd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    medpy_results[i] = {
        "hd": hd_,
        "hd95": hd95_,
        "assd": assd_,
    }
medpy_costs = time.time() - t
print(medpy_results)
print(f"Distances: {medpy_costs / mikan_costs :.2f}x faster")

{'1': {'hd': 23.23409080505371, 'hd95': 1.1611101627349854, 'assd': 0.46196645498275757}, '2': {'hd': 41.99140930175781, 'hd95': 1.5640751123428345, 'assd': 0.603213906288147}, '3': {'hd': 30.292224884033203, 'hd95': 1.5, 'assd': 0.5346971750259399}, '4': {'hd': 110.12802124023438, 'hd95': 1.1611051559448242, 'assd': 0.5072747468948364}, '5': {'hd': 92.80791473388672, 'hd95': 1.4820098876953125, 'assd': 0.5352650284767151}}
Mikan has calculated distance and cost 11.04 s.
Let's waiting for medpy, be patient for a while...
{'1': {'hd': 23.23409080505371, 'hd95': 1.1611101627349854, 'assd': 0.46196645498275757}, '2': {'hd': 41.99140930175781, 'hd95': 1.5640751123428345, 'assd': 0.603213906288147}, '3': {'hd': 30.292224884033203, 'hd95': 1.5, 'assd': 0.5346971750259399}, '4': {'hd': 110.12802124023438, 'hd95': 1.1611051559448242, 'assd': 0.5072747468948364}, '5': {'hd': 92.80791473388672, 'hd95': 1.4820098876953125, 'assd': 0.5352650284767151}}
Distances: 165.36x faster
